In [1]:
import math
import numpy as np
import pandas as pd
from task import Task
from component import Component
from core import Core
from system import System
from simulator import Simulator

## Read csv

In [2]:
large_test_arch_path = "../DRTS_Project-Test-Cases/4-large-test-case/architecture.csv"
large_test_budgets_path = "../DRTS_Project-Test-Cases/4-large-test-case/budgets.csv"
large_test_tasks_path = "../DRTS_Project-Test-Cases/4-large-test-case/tasks.csv"

## Test System class

In [3]:
large_system = System(tasks_file=large_test_tasks_path, arch_file=large_test_arch_path, budgets_file=large_test_budgets_path)
large_system.tasks, large_system.components, large_system.cores

loaded dataframes: 
  core_id  speed_factor scheduler
0  Core_1          0.54       EDF
1  Core_2          0.70       EDF
2  Core_3          0.74        RM
         component_id scheduler  budget  period core_id  priority
0       Camera_Sensor        RM       4      11  Core_1       NaN
1     Image_Processor       EDF       2       7  Core_1       NaN
2    Bitmap_Processor        RM       1       7  Core_1       NaN
3        Lidar_Sensor        RM       1       3  Core_2       NaN
4        Control_Unit       EDF       4       6  Core_2       NaN
5          GPS_Sensor        RM       3      13  Core_3       1.0
6  Communication_Unit        RM       2       4  Core_3       0.0
   task_name  wcet  period        component_id  priority
0     Task_0     4     400       Camera_Sensor       3.0
1     Task_1     8     600       Camera_Sensor       4.0
2     Task_2     3      75       Camera_Sensor       0.0
3     Task_3     2     100       Camera_Sensor       1.0
4     Task_4     2     150     

({'Task_0': <task.Task at 0x7011d4256530>,
  'Task_1': <task.Task at 0x7011d4256f50>,
  'Task_2': <task.Task at 0x7011d4256a10>,
  'Task_3': <task.Task at 0x7011d4256f80>,
  'Task_4': <task.Task at 0x7011d4257160>,
  'Task_5': <task.Task at 0x7011d4256fb0>,
  'Task_6': <task.Task at 0x7011d4256770>,
  'Task_7': <task.Task at 0x7011d4256e90>,
  'Task_8': <task.Task at 0x7011d42550c0>,
  'Task_9': <task.Task at 0x7011d42564d0>,
  'Task_10': <task.Task at 0x7011d4256ad0>,
  'Task_11': <task.Task at 0x7011d4256ce0>,
  'Task_12': <task.Task at 0x7011d4256d40>,
  'Task_13': <task.Task at 0x7011d4256bf0>,
  'Task_14': <task.Task at 0x7011d4256c50>,
  'Task_15': <task.Task at 0x7011d4257280>,
  'Task_16': <task.Task at 0x7011d42571c0>,
  'Task_17': <task.Task at 0x7011d4257490>,
  'Task_18': <task.Task at 0x7011d42574c0>,
  'Task_19': <task.Task at 0x7011d4257460>,
  'Task_20': <task.Task at 0x7011d42572e0>,
  'Task_21': <task.Task at 0x7011d4257310>,
  'Task_22': <task.Task at 0x7011d4257340>

In [4]:
for core_name, core in large_system.cores.items():
    print(core_name)
    for component in core.components:
        print(component)

Core_1
Component(id=Camera_Sensor, scheduler=RM, budget=4, period=11, core_id=Core_1, priority=0.09090909090909091, tasks=5, bdr_alpha=0.36, bdr_delta=14.00)
Component(id=Image_Processor, scheduler=EDF, budget=2, period=7, core_id=Core_1, priority=0.14285714285714285, tasks=3, bdr_alpha=0.29, bdr_delta=10.00)
Component(id=Bitmap_Processor, scheduler=RM, budget=1, period=7, core_id=Core_1, priority=0.14285714285714285, tasks=4, bdr_alpha=0.14, bdr_delta=12.00)
Core_2
Component(id=Lidar_Sensor, scheduler=RM, budget=1, period=3, core_id=Core_2, priority=0.3333333333333333, tasks=4, bdr_alpha=0.33, bdr_delta=4.00)
Component(id=Control_Unit, scheduler=EDF, budget=4, period=6, core_id=Core_2, priority=0.16666666666666666, tasks=6, bdr_alpha=0.67, bdr_delta=4.00)
Core_3
Component(id=GPS_Sensor, scheduler=RM, budget=3, period=13, core_id=Core_3, priority=1.0, tasks=2, bdr_alpha=0.23, bdr_delta=20.00)
Component(id=Communication_Unit, scheduler=RM, budget=2, period=4, core_id=Core_3, priority=0.

In [5]:
for component_name, component in large_system.components.items():
    print(f"{component_name}")
    for task in component.tasks:
        print(task)

Camera_Sensor
task Task_0 with wcet:4, period:400, component_id:Camera_Sensor, priority3.0
task Task_1 with wcet:8, period:600, component_id:Camera_Sensor, priority4.0
task Task_2 with wcet:3, period:75, component_id:Camera_Sensor, priority0.0
task Task_3 with wcet:2, period:100, component_id:Camera_Sensor, priority1.0
task Task_4 with wcet:2, period:150, component_id:Camera_Sensor, priority2.0
Image_Processor
task Task_5 with wcet:2, period:50, component_id:Image_Processor, priority0.02
task Task_6 with wcet:4, period:80, component_id:Image_Processor, priority0.0125
task Task_7 with wcet:2, period:200, component_id:Image_Processor, priority0.005
Bitmap_Processor
task Task_8 with wcet:6, period:300, component_id:Bitmap_Processor, priority1.0
task Task_9 with wcet:3, period:600, component_id:Bitmap_Processor, priority2.0
task Task_10 with wcet:3, period:75, component_id:Bitmap_Processor, priority0.0
task Task_11 with wcet:3, period:900, component_id:Bitmap_Processor, priority3.0
Lidar_S

In [6]:
def half_half_algorithm(budget:float, period:float):
    """
    Implements the Half-Half Algorithm for real-time systems.
    This algorithm calculates two parameters, alpha and delta, based on 
    the given budget and period. These parameters can be used for 
    scheduling or resource allocation in real-time systems.
    Parameters:
    -----------
    budget : float
        The allocated execution time or computational budget.
    period : float
        The time period over which the budget is allocated.
    Returns:
    --------
    alpha : represents the utilization factor, calculated as the ratio 
        of budget to period.
    delta : represents a derived parameter, calculated as twice the 
        difference between the period and the budget.
    """
    alpha = budget / period
    delta = 2 * (period - budget)
    return alpha, delta
half_half_algorithm(84, 84)

(1.0, 0)

Makes sense because this task takes up all the processing power with budget = 84 and period = 84

## Test simulator class

In [7]:
large_simulator = Simulator(large_system)
large_simulator.run_simulation()

Running simulation until hyperperiod: 7200
Active components at time 0: ['Camera_Sensor', 'Image_Processor', 'Bitmap_Processor']
Scheduled components at time 0: ['Image_Processor', 'Bitmap_Processor', 'Camera_Sensor']
Processing component: Component(id=Image_Processor, scheduler=EDF, budget=2, period=7, core_id=Core_1, priority=0.14285714285714285, tasks=3, bdr_alpha=0.29, bdr_delta=10.00)
Component Image_Processor: period is given by bdr_delta / 2 = 5.0, budget = 1.4285714285714284
Active components at time 0: ['Lidar_Sensor', 'Control_Unit']
Scheduled components at time 0: ['Lidar_Sensor', 'Control_Unit']
Processing component: Component(id=Lidar_Sensor, scheduler=RM, budget=1, period=3, core_id=Core_2, priority=0.3333333333333333, tasks=4, bdr_alpha=0.33, bdr_delta=4.00)
Component Lidar_Sensor: period is given by bdr_delta / 2 = 2.0, budget = 0.6666666666666666
Active components at time 0: ['GPS_Sensor', 'Communication_Unit']
Scheduled components at time 0: ['GPS_Sensor', 'Communicat

In [8]:
for component_name, component in large_system.components.items():
    print(f"{component_name}")
    for task in component.tasks:
        print(task.response_times)

Camera_Sensor
[334.22222222222246]
[305.814814814815]
[]
[62.6296296296299]
[51.92592592592621]
Image_Processor
[18.703703703703702, 19.81481481481481]
[51.11111111111111]
[]
Bitmap_Processor
[248.93650793650804]
[159.8253968253969]
[]
[118.2698412698413]
Lidar_Sensor
[47.428571428571104]
[]
[167.42857142857136]
[132.85714285714292]
Control_Unit
[143.9047619047612, 144.47619047618912, 140.57142857142912, 144.5714285714314, 140.5714285714298, 141.14285714285393, 143.90476190475442, 144.47619047617854, 143.90476190474578, 147.90476190474737, 143.90476190475465, 144.47619047619037, 140.57142857143526, 141.14285714287098, 140.57142857144936, 144.5714285714571, 140.57142857146437, 141.14285714290008, 140.57142857147846, 141.14285714290872, 140.57142857146573, 144.57142857145118, 140.57142857143845, 141.14285714285325, 143.90476190474237, 144.47619047615626, 143.90476190471327, 147.90476190469963, 143.9047619046869, 144.47619047610078, 143.90476190465688, 144.47619047607168, 143.904761904628

In [9]:
large_simulator.report("result_large.csv")

Generated sim results
